# ROS Manipulation in 5 Days

## Unit 4: Adding Perception to Motion Planning

<p style="background:green;color:white;">**SUMMARY**</p>

Estimated time of completion: <b>2h</b><br><br>
This unit will show you how to perform motion planning with Python. By completing this unit, you will be able to create a Python program that performs motion planning on your robot.

<p style="background:green;color:white;">**END OF SUMMARY**</p>

In the previous chapter, you saw that you can plan and execute trajectories for your robot using Python code. But... you weren't taking into account Perception, were you?
<br><br>
Usually, you will want to take the data from a 3D vision sensor into account; for instance, a Kinect camera. This will give you real-time information about the environment, and will allow you to plan more realistic motions, introduing any change that the environment suffers. So, in this chapter, we are going to learn how you can add a 3D vision sensor to MoveIt in order to perform vision-assisted Motion Planning!

But, first of all, let's make some changes in the current simulation in order to be able to better work with Perception. For that, just follow the instructions described in the next exercise!

<p style="background:#EE9023;color:white;">**Exercise 5.1**</p>
<br>
a) Create a new file named <i><b>table.urdf</b></i> in your workspace. Paste the following code into that file.

In [ ]:
<robot name="simple_box">
  <link name="my_box">
    <inertial>
      <origin xyz="0 0 0.0145"/>
      <mass value="0.1" />
      <inertia  ixx="0.0001" ixy="0.0"  ixz="0.0"  iyy="0.0001"  iyz="0.0"  izz="0.0001" />
    </inertial>
    <visual>
      <origin xyz="-0.23 0 0.215"/>
      <geometry>
        <box size="0.47 0.46 1.3"/>
      </geometry>
    </visual>
    <collision>
      <origin xyz="-0.23 0 0.215"/>
      <geometry>
        <box size="0.47 0.46 1.3"/>
      </geometry>
    </collision>
  </link>
  <gazebo reference="my_box">
    <material>Gazebo/Wood</material>
  </gazebo>
  <gazebo>
    <static>true</static>
  </gazebo>
</robot>

b) Execute the following command in order to spawn an object right in front of the Fetch robot.

In [ ]:
rosrun gazebo_ros spawn_model -file /home/user/catkin_ws/src/table.urdf -urdf -x 1 -model my_object

<img src="img/add_object.png" width="300" />

c) Execute the following command in order to move the Fetch robot's head, so that it points to the new spawned object.

In [ ]:
roslaunch fetch_gazebo_demo move_head.launch

<img src="img/head.png" width="300" />

<p style="background:red;color:white;">**IMPORTANT NOTE**</p>
<br>
Once the robot is set up, like in the image above, you must stop the command launched by pressing <b>Ctrl+C</b> in the webshell from which it was launched.

<p style="background:red;color:white;">**IMPORTANT NOTE**</p>

d) Launch RViz and add the corresponding element in order to visualize the PointCloud of the camera.

<p style="background:#AE0202;color:white;">**Expected Result for Exercise 5.1**</p>

<img src="img/rviz_object.png" width="700" />

<p style="background:#EE9023;color:white;">**End of Exercise 5.1**</p>

Great! Now you have already created the appropriate environment in order to work with Perception. So... let's now see how we can add Perception to everything we've learned about Motion Planning in the previous chapter!

## Adding Perception to MoveIt

In order to be able to add a sensor to the MoveIt package that you created in the previous chapter, you'll need to do some modifications inside the package, of course. In order to see exactly what you need to do, just follow the next exercise.

<p style="background:#EE9023;color:white;">**Exercise 5.2**</p>
<br>
a) First of all, you'll need to create a new file inside the <i><b>config</b></i> folder, named <i><b>sensors_rgbd.yaml</b></i>. Inside that file, copy the following contents:


In [ ]:
sensors:
    - sensor_plugin: occupancy_map_monitor/PointCloudOctomapUpdater
      point_cloud_topic: /head_camera/depth_registered/points
      max_range: 5
      padding_offset: 0.01
      padding_scale: 1.0
      point_subsample: 1
      filtered_cloud_topic: output_cloud

Basically, what you are doing here is configuring the plugin that we'll use in order to interface the 3D sensor with MoveIt. The parameters that you are defining in the file are the following:

* <b>sensor_plugin</b>: This parameter specifies the name of the plugin we are using in the robot.
* <b>point_cloud_topic</b>: The plugin will listen to this topic for PointCloud data.
* <b>max_range</b>: This is the distance limit, in meters, in which any points above the range will not be used for processing.
* <b>padding_offset</b>: This value will be taken into account for robot links and attached objects when filtering clouds containing the robot links (self-filtering).
* <b>padding_scale</b>: This value will also be taken into account while self-filtering.
* <b>point_subsample</b>: If the update process is slow, points can be subsampled. If we make this value greater than 1, the points will be skipped instead of processed.
* <b>filtered_cloud_topic</b>: This is the final filtered cloud topic. We will get the processed PointCloud through this topic. It can be used mainly for debugging.

b) Next, you'll need to fill in the existing, but blank, <i><b>fetch_moveit_sensor_manager.launch.xml</b></i> file, which is located in the <i><b>launch</b></i> folder. You'll need to load the YAML file you've just created into this file.

In [ ]:
<launch>
    <rosparam command="load" file="$(find test_moveit_config)/config/sensors_rgbd.yaml" />
</launch>

<p style="color:red;"><b>NOTE:</b> Note that the content of the <b>rosparam tag</b> may vary depending on how you've named your MoveIt package. In this example command, it assumes that the package is named <b>fetch_moveit_config</b>.**</p>

c) Finally, you'll have to have a look at the <i><b>sensor_manager.launch.xml</b></i> file. It should look something like this:

In [ ]:
<launch>

  <!-- This file makes it easy to include the settings for sensor managers -->  

  <!-- Params for the octomap monitor -->
  <!--  <param name="octomap_frame" type="string" value="some frame in which the robot moves" /> -->
  <param name="octomap_resolution" type="double" value="0.025" />
  <param name="max_range" type="double" value="5.0" />

  <!-- Load the robot specific sensor manager; this sets the moveit_sensor_manager ROS parameter -->
  <arg name="moveit_sensor_manager" default="fetch" />
  <include file="$(find fetch_moveit_config)/launch/$(arg moveit_sensor_manager)_moveit_sensor_manager.launch.xml" />
  
</launch>

<p style="color:red;"><b>NOTE:</b> Note that the content of the <b>include tag</b> may vary depending on how you've named your MoveIt package. In this example command, it assumes that the package is named <b>fetch_moveit_config</b>.**</p>

d) Now, you can launch the MoveIt RViz environment again, and you'll see a PointCloud in the scene, showing what the robot is visualizing.

<p style="background:#AE0202;color:white;">**Expected Result for Exercise 5.2**</p>

<img src="img/moveit_object.png" width="800" />

<p style="background:#EE9023;color:white;">**End of Exercise 5.2**</p>

Interesting, right? But how does this work? What's going on internally? Let's explain it a little bit.
<br><br>
Basically, you are using a plugin (<b>PointCloudUpdater</b>), that brings the simulated PointCloud obtained from the camera that is placed in Fetch's head, into the MoveIt planning scene.
<br><br>
The robot environment is mapped as an octree representation, which can be built using a library called OctoMap. The OctoMap is incorporated as a plugin in MoveIt (called the <b>Occupany Map Updator</b> plugin), which can update octree from different kinds of sensor inputs, such as PointClouds and depth images from 3D vision sensors.
<br><br>
Currently, there are the following plugins for handling 3D data in MoveIt:

* <b>PointCloud Occupancy Map Updater</b>: This plugin can take input in the form of PointClouds (sensor_msgs/PointCloud2). This is the one you are using in this chapter.
* <b>Depth Image Occupancy Map Updater</b>: This plugin can take input in the form of input depth images (sensor_msgs/Image).

So now you're getting real-time data from the robot's environment. Do you think this will affect the Motion Plans that are calculated in any way? Let's do the following exercise in order to check that!

<p style="background:#EE9023;color:white;">**Exercise 5.3**</p>
<br>
a) Launch the MoveIt RViz environment with Perception, if you don't have it started.

b) In the motion screen, select the <b>start</b> Pose that you created in the previous chapter, and plan a trajectory to go there.

<img src="img/plan_start.png" width="400" />

c) Remove the object of the simulation by right-clicking on it, and selecting the <b>Delete</b> option.

<img src="img/delete_object.png" width="400" />

d) Plan a trajectory to the <b>start</b> position again, and check if there is any difference from the previous trajectory.

<p style="background:#EE9023;color:white;">**End of Exercise 5.3**</p>

That's right! The calculated Motion Plans will be different, depending on how the planning scene is. Makes sense, right?

<p style="background:#EE9023;color:white;">**Exercise 5.4**</p>

a) Modify the URDF file of the object that you are spawning a little bit, and try to spawn it in different positions.

b) Keep on performing Motion Planning with the object in different positions and with different shapes, and see how it affects the trajectories that are planned. You can also execute these trajectories, if you want.

<p style="background:#EE9023;color:white;">**End of Exercise 5.4**</p>

Until now, you've been working using the PointCloud that is generated by the 3D sensor that is incorporated in the Fetch robot. But, we've previously seen that this is not the only way that we can add Perception to MoveIt, right? 
<br><br>
By doing the following exercise, let's see how we can also add Perception without using the PointCloud.

<p style="background:#EE9023;color:white;">**Exercise 5.5**</p>

a) Make sure that there is an object in front of the robot. If there isn't, execute the following command in order to spawn another object right in front of the Fetch robot.

In [ ]:
rosrun gazebo_ros spawn_model -file /home/user/catkin_ws/src/object.urdf -urdf -x 1 -model my_object

b) Modify the configuration files in order to use the <b>DepthImageUpdater</b> plugin, instead of the one you are currently using. As an example, you can have a look at the following configuration file:

In [ ]:
sensors:
    - sensor_plugin: occupancy_map_monitor/DepthImageOctomapUpdater
      image_topic: /head_camera/depth_registered/image_raw
      queue_size: 5
      near_clipping_plane_distance: 0.3
      far_clipping_plane_distance: 5.0
      skip_vertical_pixels: 1
      skip_horizontal_pixels: 1
      shadow_threshold: 0.2
      padding_scale: 4.0
      padding_offset: 0.03
      filtered_cloud_topic: output_cloud

c) Launch the whole environment again and plan a trajectory to check if it is detecting the environment correctly.

<p style="background:#EE9023;color:white;">**End of Exercise 5.5**</p>

<p style="background:red;color:white;">**IMPORTANT NOTE**</p>
<br>
Once you are done with this chapter, make sure to <b>DELETE</b> the object you've added, and return the robot to the <b>home</b> position.

<p style="background:red;color:white;">**IMPORTANT NOTE**</p>

And that's it! You have finished this chapter! I really hope that you enjoyed it and, most of all, have learned a lot! Now, if you want to learn how you can perform Grasping, just go to the next chapter!